# Tables

In [ ]:
import os
import warnings
import pandas as pd
from tableone import TableOne

In [ ]:
# Import dataset
df = pd.read_excel("dataset_fda_devices.xlsx")
df["Recall"] = df["Recall"].replace({"no": "Available", "yes": "Recalled"})

In [ ]:
df["Summary Malfunction Reporting"] = df["Summary Malfunction Reporting"].replace({"no": "ineligible"}).apply(str.capitalize)
df["Clinical Performance Study"] = df["Clinical Performance Study"].apply(lambda x: "Missing information" if x not in ["yes", "no"] else x.capitalize())
for col in ["DICOM compliance", "CP results sex available", "CP results age available"]:
    df[col] = df[col].apply(lambda x: "Missing information" if x != "yes" else "Yes")

In [ ]:
# Dates to datetime format
date_columns = [col for col in df.columns if "Date" in col]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df["Decision Time"] = (df["Decision Date"] - df["Date Received"]).dt.days
df["Recall Time"] = (df["FDA Recall Posting Date"] - df["Date of Final Decision"]).dt.days

In [ ]:
panels = ["Anesthesiology", "Cardiovascular", "Gastroenterology/Urology", "Hematology", "Neurology", "Ophthalmic", "Radiology"]
df["Panel"] = df["Panel (Lead)"].apply(lambda x: "Other" if x not in panels else x)

In [ ]:
warnings.filterwarnings('ignore')
columns = ["Geographic area", "Panel", "Submission Type", "Eligible for Third Party Review", "Summary Malfunction Reporting",
           "Implanted Device", "Physical State", "DICOM compliance", "Clinical Performance Study", "Decision Time", "Recall Time"]
non_normal = ["Decision Time", "Recall Time"]
categorical = [col for col in columns if col not in non_normal]
for column in columns:
    if df[column].dtype =='0':
        df[column] = df[column].apply(str.capitalize)
table_all = TableOne(df, columns=columns, categorical=categorical, nonnormal=non_normal, groupby="Recall", missing=False, decimals=0)
os.makedirs("tables", exist_ok=True)
table_all.to_excel(f"tables/Table1_1.xlsx")
print(table_all.tabulate(tablefmt="fancygrid"))

In [ ]:
df_cp = df[df["Clinical Performance Study"] == "Yes"].copy()
columns = ["CP Study-type", "CP results sex available", "CP results age available"]
table_all = TableOne(df_cp, columns=columns, categorical=columns, groupby="Recall", missing=False, decimals=0)
os.makedirs("tables", exist_ok=True)
table_all.to_excel(f"tables/Table1_2.xlsx")
print(table_all.tabulate(tablefmt="fancygrid"))